In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from rag.pdf import PDFRetrievalChain
from langchain_teddynote.tools.tavily import TavilySearch

from typing import List, Literal
from typing_extensions import TypedDict, Annotated
from pydantic import BaseModel, Field
from langchain_core.documents import Document

In [3]:
class GraphState(TypedDict):
    """
    그래프의 상태를 나타내는 데이터 모델

    Attributes:
        question: 질문
        generation: LLM 생성된 답변
        documents: 도큐먼트 리스트
    """

    question: Annotated[str, "User question"]
    generation: Annotated[str, "LLM generated answer"]
    documents: Annotated[List[str], "List of documents"]

### 질문 라우팅 노드

In [4]:
class RouteQuery(BaseModel):    
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )


llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

# 구조화된 출력을 결과로 가젼온다 (vectorstore, web_search)
structured_llm_router = llm.with_structured_output(RouteQuery)

# 시스템 메시지
system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to DEC 2023 AI Brief Report(SPRI) with Samsung Gause, Anthropic, etc.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""

# 프롬프트
route_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system), 
        ('human', '{question}')
    ]
)

question_router = route_prompt | structured_llm_router

In [5]:
# 질문 라우팅 노드
def route_question(state: GraphState):
    print('\n==========================================================================================')
    print('===== [ROUTE QUESTION] route_question() 함수 =====')

    # 사용자 질문
    question = state['question']

    # 질문 라우팅
    source = question_router.invoke({'question': question})

    if source.datasource == "web_search":
        print("==== [ROUTE QUESTION TO WEB SEARCH] ====")
        print('==========================================================================================\n')
        return "web_search"
    elif source.datasource == "vectorstore":
        print("==== [ROUTE QUESTION TO VECTORSTORE] ====")
        print('==========================================================================================\n')
        return "vectorstore"

In [6]:
state = GraphState(
    question='삼성전자가 만든 생성형 AI 이름은?',
    generation = '', 
    documents = []
)

state

{'question': '삼성전자가 만든 생성형 AI 이름은?', 'generation': '', 'documents': []}

In [7]:
route_question(state)


===== [ROUTE QUESTION] route_question() 함수 =====
==== [ROUTE QUESTION TO VECTORSTORE] ====



'vectorstore'

### 웹 검색 노드

In [10]:
from langchain_teddynote.tools.tavily import TavilySearch

web_search_tool = TavilySearch(max_results=2)

In [11]:
# 웹 검색 노드
def web_search(state):
    print('\n==========================================================================================')
    print('===== [WEB SEARCH] web_search() 함수 =====')

    # 사용자 질문
    question = state['question']
    
    print(f"question: ")
    print(question)
    print()

    # 웹 검색 수행
    web_results = web_search_tool.invoke({"query": question})

    print('web_results: ')
    print(web_results)
    print()

    web_results_docs = []
    
    for result in web_results:
        doc = Document(
            page_content=result["content"],
            metadata={"source": result["url"]}
        )
        web_results_docs.append(doc)

    print(web_results_docs)
    print('==========================================================================================\n')

    return {"documents": web_results_docs}

In [12]:
web_search(state)


===== [WEB SEARCH] web_search() 함수 =====
question: 
삼성전자가 만든 생성형 AI 이름은?

web_results: 
[{'title': "삼성전자 생성형 Ai 이름은 '삼성 가우스' '가이스' - 파이낸셜뉴스", 'url': 'https://www.fnnews.com/news/202309031940191378', 'content': '삼성전자 생성형 AI 이름은 \'삼성 가우스\' \'가이스\' - 파이낸셜뉴스 삼성전자 생성형 AI 이름은 \'삼성 가우스\' \'가이스\' 삼성전자가 자체 생성형 인공지능(AI) 서비스 상표권을 출원하며 연내 상용화를 위해 속도를 내고 있다. 전경훈 삼성전자 DX부문 최고기술책임자(CTO) 겸 삼성리서치장(사장)은 최근 사내 타운홀 미팅에서 자체 생성형 AI 개발 방향성을 코딩 지원, 문서 요약, 이메일 전송 등 사내 생산성 향상과 삼성 디바이스 탑재 등 두 가지로 설명한 바 있다. 가출 후 2주 만에 실종된 남편, 숨진 채 발견 [속보]김현태 707단장 "곽종근 자수서에 \'국회의원\' \'끌어내라\' 단어 없다" [속보]김현태 707단장 "尹지시가 단전 배경이란 건 내용 안 맞아" 현대차 크레타 EV, 인도서 돌풍…2주 만에 1700대 판매 가출 후 2주 만에 실종된 남편, 숨진 채 발견 "청소 아줌마, 당 떨어진다며 3만원 양갱 몰래 먹어…따지니 \'엄마뻘한테?\'" [어떻게 생각하세요] "9개월 만에 20kg 감량" 20대 여성이 밝힌 놀라운 비법은', 'score': 0.93206495, 'raw_content': 'Published Time: 2023-09-03T19:40:15 +0900\n삼성전자 생성형 AI 이름은 \'삼성 가우스\' \'가이스\' - 파이낸셜뉴스\n파이낸셜뉴스>\n전체메뉴 검색\n\n구독신청\nFamily Site\n\n금융·증권\n\n금융\n증권\n\n부동산\n\n정책\n건설\n철도·항공 ·선박\n부동산 일반\n\n산업·IT\n\n산업\n통신·방송\n게

{'documents': [Document(metadata={'source': 'https://www.fnnews.com/news/202309031940191378'}, page_content='삼성전자 생성형 AI 이름은 \'삼성 가우스\' \'가이스\' - 파이낸셜뉴스 삼성전자 생성형 AI 이름은 \'삼성 가우스\' \'가이스\' 삼성전자가 자체 생성형 인공지능(AI) 서비스 상표권을 출원하며 연내 상용화를 위해 속도를 내고 있다. 전경훈 삼성전자 DX부문 최고기술책임자(CTO) 겸 삼성리서치장(사장)은 최근 사내 타운홀 미팅에서 자체 생성형 AI 개발 방향성을 코딩 지원, 문서 요약, 이메일 전송 등 사내 생산성 향상과 삼성 디바이스 탑재 등 두 가지로 설명한 바 있다. 가출 후 2주 만에 실종된 남편, 숨진 채 발견 [속보]김현태 707단장 "곽종근 자수서에 \'국회의원\' \'끌어내라\' 단어 없다" [속보]김현태 707단장 "尹지시가 단전 배경이란 건 내용 안 맞아" 현대차 크레타 EV, 인도서 돌풍…2주 만에 1700대 판매 가출 후 2주 만에 실종된 남편, 숨진 채 발견 "청소 아줌마, 당 떨어진다며 3만원 양갱 몰래 먹어…따지니 \'엄마뻘한테?\'" [어떻게 생각하세요] "9개월 만에 20kg 감량" 20대 여성이 밝힌 놀라운 비법은'),
  Document(metadata={'source': 'https://www.bizhankook.com/bk/article/26352'}, page_content="생성형 ai 선구자, 챗지피티(chatgpt)를 만든 오픈ai도 생성형 ai와 관련해 'chatgpt', 'gpt', 'gpt-4', 'gpt-5'의 상표를 국내에 출원했다. 생성형 ai 상표의 경우 주로 제9류인 ai 소프트웨어 및 제42류의 ai 소프트웨어 개발업 등을 지정해 상표출원 하게 된다.")]}

### 문서 검색 노드 : 사용자 질문에 대한 PDF 문서를 검색하는 PDF 기반 문서 검색기 (Retrieval chain)

In [13]:
# PDF 문서를 로드
pdf = PDFRetrievalChain(['data/SPRI_AI_Brief_2023년12월호_F.pdf']).create_chain()

pdf_retriever = pdf.retriever       # retriever 생성
pdf_chain = pdf.chain               # pdf retriever chain 생성

In [14]:
# 문서 검색 노드
def retrieve(state: GraphState):
    print('\n==========================================================================================')
    print('===== [RETRIEVE] retieve() 노드 실행 =====')

    # 사용자 질문
    question = state['question']        

    # 문서 검색 (pdf_retriever 에 사용자 입력을 인자로 넣어서 문서를 검색)
    documents = pdf_retriever.invoke(question)
    
    print(f'문서 검색 결과: ')
    print(documents)
    print('==========================================================================================\n')
    
    return {'documents': documents}

In [15]:
answer = retrieve(state)                        # 사용자 질문에 대해서 관련 문서를 검색한다.
state['documents'] = answer['documents']


===== [RETRIEVE] retieve() 노드 실행 =====
문서 검색 결과: 
[Document(id='74eb8859-4cdc-4166-b090-f3b02c3bbdcd', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11\n▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12'), Document(id='6d08664d-42c5-4a54-91a5-86954096bab4', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', '

In [16]:
state

{'question': '삼성전자가 만든 생성형 AI 이름은?',
 'generation': '',
 'documents': [Document(id='74eb8859-4cdc-4166-b090-f3b02c3bbdcd', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11\n▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12'),
  Document(id='6d08664d-42c5-4a54-91a5-86954096bab4', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Han

### 사용자 질문에 대한 문서 검색 결과와 사용자 질문의 관련성 평가하는 노드 (문서 검색 평가기 노드)

In [17]:
class GradeDocuments(BaseModel):        # 문서 평가를 위한 데이터 모델
    """Binary score for relevance check on retrieved documents."""

    # 사용자 질문에 대한 pdf 문서 검색 결과를 평가하는 문서 검색 평가기(retrieval_grader)를 사용해서 
    # 사용자 질문과 검색 결과문서가 관련성이 있는지 본다

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 시스템 메시지
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

# 프롬프트 템플릿
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# 문서 검색결과 평가기
retrieval_grader = grade_prompt | structured_llm_grader

In [18]:
# 문서 검색 결과와 질문 관련성 평가 노드
def grade_documents(state: GraphState):
    print('\n==========================================================================================')
    print('===== [CHECK DOCUMENT RELEVANCE TO QUESTION] grade_documents() 함수 =====')

    # 사용자 질문
    question = state['question']  

    # 문서 검색 결과
    documents = state['documents']

    # 문서 검색 평가기를 통과한 문서들만 필터링
    filtered_docs = []

    for doc in documents:
        score = retrieval_grader.invoke({'question': question, 'document': doc.page_content})

        grade = score.binary_score

        if grade == 'yes':
            print("---GRADE: DOCUMENT RELEVANT---")     
            filtered_docs.append(doc)                   
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---") 
            continue     

    print('==========================================================================================\n')
    return {'documents': filtered_docs}

In [19]:
state['documents'] = grade_documents(state)['documents']


===== [CHECK DOCUMENT RELEVANCE TO QUESTION] grade_documents() 함수 =====
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---



In [20]:
state

{'question': '삼성전자가 만든 생성형 AI 이름은?',
 'generation': '',
 'documents': [Document(id='74eb8859-4cdc-4166-b090-f3b02c3bbdcd', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11\n▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12'),
  Document(id='6d08664d-42c5-4a54-91a5-86954096bab4', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Han

### 쿼리 재작성 (Query Rewriter) 노드

In [21]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

# Query Rewriter 프롬프트
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""

re_writer_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system), 
        ('human', 'Here is the initial question: \n\n {question} \n Formulate an improved question')
    ]
)

# Query Rewriter 생성
question_rewriter = re_writer_prompt | llm | StrOutputParser()

In [22]:
# Query Rewrite 노드
def transform_query(state: GraphState):
    print('\n==========================================================================================')
    print('===== [TRANSFORM QUERY] transform_query() 함수 =====')

    # 사용자 질문
    question = state['question']

    # 문서 검색 결과
    documents = state['documents']

    
    # 쿼리 재작성
    better_question = question_rewriter.invoke({'question': question})
    print(f"better_question: {better_question}")
    print('==========================================================================================\n')

    return {'question': better_question}

In [26]:
state['question'] = transform_query(state)['question']


===== [TRANSFORM QUERY] transform_query() 함수 =====
better_question: 삼성전자가 개발한 생성형 AI의 이름은 무엇인가요?



In [27]:
state

{'question': '삼성전자가 개발한 생성형 AI의 이름은 무엇인가요?',
 'generation': '',
 'documents': [Document(id='74eb8859-4cdc-4166-b090-f3b02c3bbdcd', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11\n▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12'),
  Document(id='6d08664d-42c5-4a54-91a5-86954096bab4', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Produce

### 사용자 질문에 대한 답변을 생성하는 노드 (답변 생성을 위한 RAG 체인을 실행하여 답변을 생성)

In [28]:
# 답변 생성을 위한 체인

template = """You are an AI assistant specializing in Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system. 
Your primary mission is to answer questions based on provided context or chat history.
Ensure your response is concise and directly addresses the question without any additional narration.

###

Your final answer should be written concisely (but include important numerical values, technical terms, jargon, and names), followed by the source of the information.

# Steps

1. Carefully read and understand the context provided.
2. Identify the key information related to the question within the context.
3. Formulate a concise answer based on the relevant information.
4. Ensure your final answer directly addresses the question.
5. List the source of the answer in bullet points, which must be a file name (with a page number) or URL from the context. Omit if the source cannot be found.

# Output Format:
[Your final answer here, with numerical values, technical terms, jargon, and names in their original language]

**Source**(Optional)
- (Source of the answer, must be a file name(with a page number) or URL from the context. Omit if you can't find the source of the answer.)
- (list more if there are multiple sources)
- ...

###

Remember:
- It's crucial to base your answer solely on the **PROVIDED CONTEXT**. 
- DO NOT use any external knowledge or information not present in the given materials.
- If you can't find the source of the answer, you should answer that you don't know.

###

# Here is the user's QUESTION that you should answer:
{question}

# Here is the CONTEXT that you should use to answer the question:
{context}

# Your final ANSWER to the user's QUESTION:"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

def format_docs(docs):
    return "\n\n".join(
        [
            f'<document><content>{doc.page_content}</content><source>{doc.metadata["source"]}</source><page>{doc.metadata["page"]+1}</page></document>'
            for doc in docs
        ]
    )

output_parser = StrOutputParser()

# RAG 체인 생성 (pdf_retriever 문서 검색기를 사용해서 얻은 문서와 사용자 질문을 넣어서 결과)
rag_chain = prompt | llm | output_parser

In [29]:
# 답변 생성 노드
def generate(state: GraphState):
    print('\n==========================================================================================')
    print('===== [GENERATE] generate() 함수 =====')

    # 사용자 질문
    question = state['question']  

    # 문서 검색 결과
    documents = state['documents']

    # RAG 실행 결과
    generation = rag_chain.invoke({'context': documents , 'question': 'question'})

    print(f'사용자 질문: {question}')
    print(f'문서 검색 결과:')
    print(documents)
    print()
    print(f'rag 실행 결과:')
    print(generation)
    print()
    print('==========================================================================================\n')    

    return {'generation': generation}

In [30]:
state['generation'] = generate(state)['generation']


===== [GENERATE] generate() 함수 =====
사용자 질문: 삼성전자가 개발한 생성형 AI의 이름은 무엇인가요?
문서 검색 결과:
[Document(id='74eb8859-4cdc-4166-b090-f3b02c3bbdcd', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11\n▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12'), Document(id='6d08664d-42c5-4a54-91a5-86954096bab4', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Pr

In [31]:
state

{'question': '삼성전자가 개발한 생성형 AI의 이름은 무엇인가요?',
 'generation': "삼성전자가 공개한 생성 AI '삼성 가우스'는 언어, 코드, 이미지의 3개 모델로 구성되어 있으며, 온디바이스에서 작동하도록 설계되어 사용자 정보 유출 위험이 없다. 이 AI는 2023년 11월 8일 '삼성 AI 포럼 2023'에서 처음 공개되었다.\n\n**Source**\n- data/SPRI_AI_Brief_2023년12월호_F.pdf (page 12)",
 'documents': [Document(id='74eb8859-4cdc-4166-b090-f3b02c3bbdcd', metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 1, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11\n▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12'),
  Document(id='6d08664d-42c5-4a54-91a5-86954096bab4',

### 실행 답변에 대한 Hallucination 평가 노드

In [ ]:
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

structured_hallucination_grader = llm.with_structured_output(GradeHallucinations)

# system
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""

# 프롬프트
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
) 

# hallucination 평가기 생성
hallucination_grader = hallucination_prompt | structured_hallucination_grader

In [33]:
class GradeAnswer(BaseModel):
    """Binary scoring to evaluate the appropriateness of answers to questions"""

    binary_score: str = Field(
        description="Indicate 'yes' or 'no' whether the answer solves the question"
    )

llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

structured_llm_grader = llm.with_structured_output(GradeAnswer)

# system
system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""

# 프롬프트 템플릿
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

# 프롬프트 템플릿과 구조화된 LLM 평가기를 결합하여 답변 평가기 생성
answer_grader = answer_prompt | structured_llm_grader

In [34]:
# Hallucination 평가 노드
def hallucination_check(state: GraphState):
    print('\n==========================================================================================')
    print('===== [CHECK hallucination] hallucination_check() 함수 =====')

    # 사용자 질문
    qestion = state['question']

    # 문서 검색 결과
    documents = state['documents']

    # 생성된 답변
    generation = state['generation']

    # 평가
    score = hallucination_grader.invoke({'documents': documents, 'generation': generation})
    grade = score.binary_score

    print(f"===== [평가 결과] =====")
    print(grade)

    if grade == 'yes':     # hallucination 없음
        print("===== [DECISION: GENERATION IS GROUNDED IN DOCUMENTS] =====")
        print("no hallucination")

        # 답변에 대한 관련성 평가
        print("===== [GRADE GENERATED ANSWER vs QUESTION] =====")

        score = answer_grader.invoke({'question': state['question'], 'generation': generation})
        grade = score.binary_score

        if grade == 'yes':          # 관련성 있는 답변
            print("==== [DECISION: GENERATED ANSWER ADDRESSES QUESTION] ====")
            print("relevant")
            print('==========================================================================================\n')
            return "relevant"
        else:                       # 관련성 없는 답변
            print("==== [DECISION: GENERATED ANSWER DOES NOT ADDRESS QUESTION] ====")
            print("not relevant")
            print('==========================================================================================\n')
            return "not relevant"

    else:                   # hallucination 있음
        print("===== [DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY] =====")
        print("hallucination")
        print('==========================================================================================\n')
        return "hallucination"

In [35]:
hallucination_check(state)


===== [CHECK hallucination] hallucination_check() 함수 =====
===== [평가 결과] =====
yes
===== [DECISION: GENERATION IS GROUNDED IN DOCUMENTS] =====
no hallucination
===== [GRADE GENERATED ANSWER vs QUESTION] =====
==== [DECISION: GENERATED ANSWER ADDRESSES QUESTION] ====
relevant



'relevant'